In [122]:
import graphlab
import random
import math
import numpy
from matplotlib import pyplot as plt
%matplotlib inline

In [123]:
#vendas = graphlab.SFrame("home_data.gl")

In [124]:
#k igual a 5
n = 5
x = graphlab.SArray([random.random() for i in range(n)]).sort()
y = x.apply(lambda x:math.sin(4*x))
random.seed(1)
e = graphlab.SArray([random.gauss(0,1.0/3.0) for i in range(n)])
y = y+e
data = graphlab.SFrame({'X1':x,'Y':y})
def polynomial_features(data,deg):
    data_copy=data.copy()
    for i in range(1,deg):
        data_copy['X'+str(i+1)]=data_copy['X'+str(i)]*data_copy['X1']
    return data_copy

In [125]:
def loo(data, deg, l2_penalty_values):
    data = polynomial_features(data, deg)
    num_folds = len(data)
    folds = graphlab.cross_validation.KFold(data,num_folds)
    l2_penalty_mse = []
    min_mse = None
    best_l2_penalty = None
    
    for l2_penalty in l2_penalty_values:
        next_mse = 0.0
        for train_set, validation_set in folds:
            model = graphlab.linear_regression.create(train_set,target='Y',
                                                      l2_penalty=l2_penalty,
                                                      validation_set=None,
                                                      verbose=False)
            
            y_test_predicted = model.predict(validation_set)
            next_mse += ((y_test_predicted-validation_set['Y'])**2).sum()
    
        next_mse = next_mse/num_folds
        l2_penalty_mse.append(next_mse)
        
    if min_mse is None or next_mse < min_mse:
        min_mse = next_mse
        best_l2_penalty = l2_penalty
        
    return l2_penalty_mse,best_l2_penalty

In [126]:
l2_penalty_values = numpy.logspace(-4, 10, num=10)

In [127]:
l2_penalty_mse,best_l2_penalty = loo(data, 5, l2_penalty_values)

In [128]:
def print_coefficients(model):
    deg = len(model.coefficients['value'])-1
    w = list(model.coefficients['value'])
    print 'Coeficientes do polinômio de grau '+str(deg)+" : "
    w.reverse()
    print numpy.poly1d(w)

In [129]:
def regressao_polinomial_cume(data, deg, l2_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg),
    target='Y', l2_penalty=l2_penalty,
    validation_set=None,verbose=False)
    return model

In [130]:
model = regressao_polinomial_cume(data,16,l2_penalty=100)

In [131]:
#print de k 5
print_coefficients(model)

Coeficientes do polinômio de grau 16 : 
         16          15          14         13          12          11
-0.5683 x  - 0.4778 x  - 0.4015 x  - 0.337 x  - 0.2825 x  - 0.2364 x 
           10          9         8          7           6           5
 - 0.1973 x  - 0.1642 x - 0.136 x - 0.1119 x - 0.09121 x - 0.07317 x
            4           3          2
 - 0.05703 x - 0.04185 x - 0.0265 x - 0.01072 x + 0.5797


In [132]:
#k igual a 10
n = 10
x = graphlab.SArray([random.random() for i in range(n)]).sort()
y = x.apply(lambda x:math.sin(4*x))
random.seed(1)
e = graphlab.SArray([random.gauss(0,1.0/3.0) for i in range(n)])
y = y+e
data = graphlab.SFrame({'X1':x,'Y':y})
l2_penalty_values = numpy.logspace(-4, 10, num=10)
l2_penalty_mse,best_l2_penalty = loo(data, 5, l2_penalty_values)
model = regressao_polinomial_cume(data,16,l2_penalty=100)
#impressão de k igual a 10
print_coefficients(model)

Coeficientes do polinômio de grau 16 : 
         16          15          14          13          12          11
-0.4359 x  - 0.3753 x  - 0.3227 x  - 0.2768 x  - 0.2367 x  - 0.2016 x 
           10          9          8           7           6           5
 - 0.1709 x  - 0.1439 x - 0.1202 x - 0.09928 x - 0.08079 x - 0.06441 x
            4           3           2
 - 0.04978 x - 0.03657 x - 0.02442 x - 0.01307 x + 0.3785
